In [1]:
!pip install scrapy
import scrapy
import re
from scrapy.crawler import CrawlerProcess
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.4 MB/s eta 0:00:00


In [2]:
class GorenjeVgradniHladilnikiSpider(scrapy.Spider):
  name="hladilniki_spider"
  allowed_domains=['si.gorenje.com']
  start_urls=['https://si.gorenje.com/izdelki/hlajenje/vgradni-hladilniki/c/gocool02?Page=10']
  custom_settings= {
      'FEEDS': {
          'VgradniHladilniki.json':{
              'format': 'json',
              'encoding': 'utf8',
              'overwrite': True
          }
      }
  }

  def parse(self, response):

        naslednje_strani = response.css('a.table-product-info__title::attr(href)').extract()
        print(naslednje_strani)
        for i in naslednje_strani:
            url = 'https://si.gorenje.com/'+i
            yield scrapy.Request(url, callback=self.parse_subpage)

  def parse_subpage(self, response):


        std = {}
        product_title = response.css('div.product-gallery__title')
        std['Ime_hladilnika'] = product_title.css('::text').extract_first()
        std['Serijska_stevilka'] = product_title.css('::attr(data-product-item-name)').extract_first()
        std['Kategorija'] = product_title.css('::attr(data-product-main-category)').extract_first()
        image_url = response.css('div.slider-picture img::attr(src)').extract_first()
        std['Slika'] = image_url

        old_price_text = response.css('div.price__old::text').extract_first().strip()
        old_price_value = old_price_text.replace('€', '').replace(',', '.').strip()
        std['Redna_Cena'] = old_price_value

        new_price_value = response.css('div.price__new::attr(val)').extract_first().strip()
        std['Trenutna_Cena'] = new_price_value

        tech_specs = response.css('div.tech-specs__item')
        for spec in tech_specs:
            label = spec.css('div.tech-specs__item-label::text').extract_first().strip()
            label = label.replace(' ', '_')
            content = spec.css('div.tech-specs__item-content::text').extract_first().strip()
            std[label] = content


        yield std



In [3]:
process = CrawlerProcess()
process.crawl(GorenjeVgradniHladilnikiSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)
2024-05-22 12:03:18 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.7, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-05-22 12:03:18 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.7, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-05-22 12:03:18 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERP

['/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI2788ROE-GDNRK5182A2-GOR/p/000000000000733715', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI2788RFF-NRKI518EA1-GOR/p/000000000000740154', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI2728RFF-NRKI2181A1-GOR/p/000000000000732565', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI2728RFF-NRKI218EA0-GOR/p/000000000000743273', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI29285-NRKI419EP1-GOR/p/000000000000741836', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HI3188RFF-RI518EA1-GOR/p/000000000000742197', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI2728RBH-NRKI218EE1-GOR/p/000000000000743384', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HTI3028RMB-RBI418EE0-GOR/p/000000000000743531', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZI2728RFF-RKI218EA0-GOR/p/000000000000743281', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZFI26282FD-NRKI418EP1-GOR/p/000000000020011296', '/izdelki/hlajenje/vgradni-hladilniki/REFRIG-HI3128RMB-RI4

DEBUG:scrapy.core.engine:Crawled (200) <GET https://si.gorenje.com//izdelki/hlajenje/vgradni-hladilniki/REFRIG-HTI13283-RBI409EP1-GOR/p/000000000000741019> (referer: https://si.gorenje.com/izdelki/hlajenje/vgradni-hladilniki/c/gocool02?Page=10)
2024-05-22 12:03:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://si.gorenje.com//izdelki/hlajenje/vgradni-hladilniki/REFRIG-HTI13283-RBI409EP1-GOR/p/000000000000741019> (referer: https://si.gorenje.com/izdelki/hlajenje/vgradni-hladilniki/c/gocool02?Page=10)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://si.gorenje.com//izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZZI18265-RFI412EP1-GOR/p/000000000000743339> (referer: https://si.gorenje.com/izdelki/hlajenje/vgradni-hladilniki/c/gocool02?Page=10)
2024-05-22 12:03:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://si.gorenje.com//izdelki/hlajenje/vgradni-hladilniki/REFRIG-HZZI18265-RFI412EP1-GOR/p/000000000000743339> (referer: https://si.gorenje.com/izdelki/hlajenje/vgradni-hla

In [4]:
df=pd.read_json("VgradniHladilniki.json")
print(df.to_string())

         Ime_hladilnika Serijska stevilka                  Kategorija                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Slika Redna_Cena  Trenutna_Cena             Družina_izdelka       Način_postavitve       Linija_izdelka Razred_energijske_učinkovitosti Razred_hrupa Barva_izdelka Barva_vrat           Material_vrat  Odpiranje_vrat                           Tečaj_vrat  Število_vrat Število_zvezdic Klimatski_razredi Skupna_prostornina Prostornina_hladilnega_prostora Prostornina_zamrzovalnega_prostora Shranjevalni_čas_ob_motnjah Ocenjena_letna_poraba_e